## Multi-Layer Perceptron, MNIST (PyTorch)

- [참고 링크](https://wikidocs.net/60324)

In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms

### 1. torchvision에서 MNIST 데이터 불러오기

- [torch.utils.data.DataLoader](https://pytorch.org/docs/stable/data.html) Documentation
  - dataset (Dataset) – dataset from which to load the data
  - batch_size (int, optional) – how many samples per batch to load (default: 1)
  - num_workers (int, optional) – how many subprocesses to use for data loading. 0 means that the data will be loaded in the main process (default: 0)

In [2]:
batch_size = 20

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



### 2. MLP 아키텍쳐 만들기

- [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) Documentation
  - in_features (int) – size of each input sample
  - out_features (int) – size of each output sample
  - bias (bool) – If set to False, the layer will not learn an additive bias (default: True)

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
  ## 신경망 정의
  def __init__(self):
    super(Net, self).__init__()

    # 이미지의 Input size가 28 pixel * 28 pixel
    self.layer0 = nn.Linear(28*28, 512)
    self.layer1 = nn.Linear(512, 10)

    self.act = nn.ReLU()

  ## 신경망에 데이터를 지나가게 하는 forward 함수를 정의 (x는 데이터를 나타냄)
  def forward(self, x):
    x = x.view(-1, 28*28) # flatten
    x = self.act(self.layer0(x))
    return x

## Initialize the NN
model = Net()
print(model)

Net(
  (layer0): Linear(in_features=784, out_features=512, bias=True)
  (layer1): Linear(in_features=512, out_features=10, bias=True)
  (act): ReLU()
)


### 3. 모델 학습하기

- Loss function
- Optimizer

In [7]:
## 랜덤 값 생성
criterion = nn.CrossEntropyLoss() # 내부적으로 소프트맥스 함수가 포함되어 있음

## 가중치 갱신 (확률적 경사하강법, SGD)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

n_epochs = 30

model.train()

for epoch in range(n_epochs):
  train_loss = 0.0 # training loss 확인하기

  for data, target in train_loader:
    optimizer.zero_grad()
    output = model(data)                      # forward pass
    loss = criterion(output, target)          # calculate loss
    loss.backward()                           # backward pass
    optimizer.step()                          # update parameter
    train_loss += loss.item() * data.size(0)  # update train_loss
  
  train_loss = train_loss / len(train_loader.dataset)
  print(f'Epoch: {epoch + 1} \tTraining Loss: {train_loss}')

Epoch: 1 	Training Loss: 0.3972972020888701
Epoch: 2 	Training Loss: 0.30130286337652556
Epoch: 3 	Training Loss: 0.2873219612624962
Epoch: 4 	Training Loss: 0.2797099689690707
Epoch: 5 	Training Loss: 0.27462694959528744
Epoch: 6 	Training Loss: 0.2708717935799311
Epoch: 7 	Training Loss: 0.2679222571979432
Epoch: 8 	Training Loss: 0.26550833153499603
Epoch: 9 	Training Loss: 0.26347405037726274
Epoch: 10 	Training Loss: 0.2617219113383908
Epoch: 11 	Training Loss: 0.2601781315811289
Epoch: 12 	Training Loss: 0.25881397563614883
Epoch: 13 	Training Loss: 0.25758982520298257
Epoch: 14 	Training Loss: 0.2564814047679926
Epoch: 15 	Training Loss: 0.2554702503818941
Epoch: 16 	Training Loss: 0.25454189258103727
Epoch: 17 	Training Loss: 0.25368482634432926
Epoch: 18 	Training Loss: 0.2528897365136848
Epoch: 19 	Training Loss: 0.25214897025687
Epoch: 20 	Training Loss: 0.251456184637694
Epoch: 21 	Training Loss: 0.2508060975166736
Epoch: 22 	Training Loss: 0.25019417655759024
Epoch: 23 	Tr

### 4. 모델 성능 테스트하기

In [8]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()

for data, target in test_loader:
  output = model(data)
  loss = criterion(output, target)
  test_loss += loss.item() * data.size(0)

  _, pred = torch.max(output, 1) # 예측된 값에서 가장 확률값이 높은 클래스 찾기
  correct = np.squeeze(pred.eq(target.data.view_as(pred)))

  for i in range(batch_size):
    label = target.data[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1

test_loss = test_loss / len(test_loader.dataset)
print(f'Test Loss: {test_loss}')

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (class_total[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.2868712410754524
Test Accuracy of     0: 98% (962/980)
Test Accuracy of     1: 98% (1117/1135)
Test Accuracy of     2: 86% (896/1032)
Test Accuracy of     3: 90% (914/1010)
Test Accuracy of     4: 91% (900/982)
Test Accuracy of     5: 91% (816/892)
Test Accuracy of     6: 93% (893/958)
Test Accuracy of     7: 89% (921/1028)
Test Accuracy of     8: 85% (831/974)
Test Accuracy of     9: 93% (943/1009)

Test Accuracy (Overall): 91% (9193/10000)
